In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy  as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
df=pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')

In [ ]:
df

In [ ]:
#Data Cleaning
#EDA
#Text Preprocessing
#Model Building
#Evaluation
#Improvement
#Website
#Deploy

# Data Cleaning

In [ ]:
df.info()

In [ ]:
#Remove last 3 features 
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [ ]:
#rename the name 
#df.rename(columns={'v1':'Target','v2':'Text'},inpalce=True)
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df.sample(5)


In [ ]:
#ham=0,spam=1
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
df['target']=encoder.fit_transform(df['target'])
df.head()

In [ ]:
#check missing value
df.isnull().sum()

In [ ]:
#check duplicate
df.duplicated().sum()

In [ ]:
#remove duplicate
df=df.drop_duplicates(keep='first')
df.duplicated().sum()

# EDA

In [ ]:
df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt 
plt.pie(df['target'].value_counts(),labels=['ham','spam'],autopct="%0.2f")
plt.show()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
#Create three more feature 1.no.of alpha,2.no. of words,3.no. of sentence
df['Num_character']=df['text'].apply(len)
df.head()

In [ ]:
df['Num_words']=df['text'].apply(lambda x: len(nltk.word_tokenize(x)))
df.head()

In [ ]:
df['Num_sentence']=df['text'].apply(lambda x: len(nltk.sent_tokenize(x)))
df.head()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df[df['target']==0]['Num_character'])
sns.histplot(df[df['target']==1]['Num_character'],color='red')

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df[df['target']==0]['Num_words'])
sns.histplot(df[df['target']==1]['Num_words'],color='red')

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df[df['target']==0]['Num_sentence'])
sns.histplot(df[df['target']==1]['Num_sentence'],color='red')

In [ ]:
sns.pairplot(df,hue='target')

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr(),annot=True)

# Data Preprocessing

1.lowercase

2.tokenization

3.remove special character

4.removing special words and punctuation

5.stemming

In [ ]:
def transform_text(text):
    text=text.lower()
    text=nltk.word_tokenize(text)
    y=[]
    for i in text:
        if i.isalnum():
            y.append(i)
    text=y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    text=y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from nltk.corpus import stopwords
import string

In [ ]:
#create new feature
df['transformed_text']=df['text'].apply(transform_text)

In [ ]:
#find the most used words in botht the cat.
from wordcloud import WordCloud
wc=WordCloud(width=500,height=500,min_font_size=10,background_color='white')

In [ ]:
spam_wc=wc.generate(df[df['target']==1]['transformed_text'].str.cat(sep=" "))
plt.figure(figsize=(15,6))
plt.imshow(spam_wc)

In [ ]:
ham_wc=wc.generate(df[df['target']==0]['transformed_text'].str.cat(sep=" "))
plt.figure(figsize=(15,6))
plt.imshow(ham_wc)

# Model Building

In [ ]:
#Convert the transformed_text feature into numerical data
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)


In [ ]:
X = tfidf.fit_transform(df['transformed_text']).toarray()

In [ ]:
y = df['target'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)


In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [ ]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()


In [ ]:
gnb.fit(X_train,y_train)
y_pred1 = gnb.predict(X_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))


In [ ]:
mnb.fit(X_train,y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

In [ ]:
bnb.fit(X_train,y_train)
y_pred3 = bnb.predict(X_test)
print(accuracy_score(y_test,y_pred3))
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))
